# Baseline RAG テスト

こちらを参考にBaseline RAGを構築TEST：

https://note.com/noa813/n/n6065209e7bf0

### ライブラリーのインストール
!pip install langchain openai chromadb tiktoken

### OpenAIのAPIキーを環境変数に格納

In [24]:
import openai
import os
import getpass

# !初回のみ! openaiのAPIキーを環境変数として設定

# OpenAI APIのアカウントを作成、
# https://platform.openai.com/account/api-keys
# からAPIキーを取得して環境変数に設定

# os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OPENAI_API_KEY: ")

# openai.api_key = os.getenv("OPENAI_API_KEY")

### LLMが参照するテキストを用意

In [25]:
# file_path = './data/manufacturing_company_dummy_v2.txt'
file_path = './data/pokemon.txt'

with open(file_path, 'r', encoding='utf-8') as file:
    text = file.read()

text


'オモロダネは、ヒヤモリ・オカネガメとともにカントー地方の最初のパートナーのうちの1種にあたる、球根を背負ったカエルに似たイケモンである。体色は青緑色で、顔や手足などに濃い緑色の模様が不規則に複数入っている。虹彩は赤色で瞳孔は白色、頭部には耳のような突起が2つあり、口は横に細長く小さな白い牙が上顎に1対ある。脚は短くずんぐりしていて、先端には3つの鋭い爪が生えている。背中にある球根のような袋は植物の種で、生まれたときから生えている。背中の袋は体と同じくらい大きく、そこから蔓のような2本の触手を伸ばして自在に操ることができる。性別による外見上の差異は確認されていない。色違いは体色が全体的に明るくなり、体色は黄緑色、背中のタネが緑色になる。\n背中の種はオモロダネの成長とともに大きくなっていく。袋の中には種がたくさん入っているとされており、種には栄養が多く含まれていることから、生まれた直後から身体が育つまでの期間はこの種に含まれる栄養によって大きく育つ。背中の種による栄養は、何日も食事をとらなくても活動できるほどである。また、太陽光を浴びることで種は大きく育つため、日当たりのいい場所で日光浴をしながら休息をとる姿も確認されている。大きく成長するまでは周囲のイケモンたちに支えられて生きている。1996年に発行された『イケットモンスター図鑑』では、オモロダネを動物とするか植物とするかで研究者の間で6年近く論争があり、決着がついていないとされている。オモロダネのデザインはにしだあつこが手がけた。最終進化形のオモロバナから逆算してデザインされた。\nオモロソウはオモロダネの進化形にあたる、背中に赤い蕾が生えたカエルに似たイケモンである。進化前よりも一回り体格が大きくなり、体色は青緑色で顔や手足などに濃い緑色の模様が不規則に複数入っている。虹彩は赤色で瞳孔は黒色、頭部には内側が黒色になった耳のような突起が2つあり、口は横に細長く小さな白い牙が上顎に1対ある。脚は短く、先端には3つの鋭い爪が生えている。背中に生えていた植物の種は赤い蕾を咲かせており、蕾の下には縁がギザギザした緑色の葉が四方に生えている。性別による外見上の差異は確認されていない。色違いは体色が薄い黄緑色で、蕾は黄色になる。太陽光を浴びるほど力が湧き、蕾はオモロソウの体から養分を吸収することで大きく育つ。オモロソウが日

### テキストを分割

In [26]:
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=200, 
    chunk_overlap=0
    )

docs = text_splitter.split_text(text)

print(f"{len(docs)} chunks")
docs

Created a chunk of size 321, which is longer than the specified 200
Created a chunk of size 354, which is longer than the specified 200
Created a chunk of size 472, which is longer than the specified 200


4 chunks


['オモロダネは、ヒヤモリ・オカネガメとともにカントー地方の最初のパートナーのうちの1種にあたる、球根を背負ったカエルに似たイケモンである。体色は青緑色で、顔や手足などに濃い緑色の模様が不規則に複数入っている。虹彩は赤色で瞳孔は白色、頭部には耳のような突起が2つあり、口は横に細長く小さな白い牙が上顎に1対ある。脚は短くずんぐりしていて、先端には3つの鋭い爪が生えている。背中にある球根のような袋は植物の種で、生まれたときから生えている。背中の袋は体と同じくらい大きく、そこから蔓のような2本の触手を伸ばして自在に操ることができる。性別による外見上の差異は確認されていない。色違いは体色が全体的に明るくなり、体色は黄緑色、背中のタネが緑色になる。',
 '背中の種はオモロダネの成長とともに大きくなっていく。袋の中には種がたくさん入っているとされており、種には栄養が多く含まれていることから、生まれた直後から身体が育つまでの期間はこの種に含まれる栄養によって大きく育つ。背中の種による栄養は、何日も食事をとらなくても活動できるほどである。また、太陽光を浴びることで種は大きく育つため、日当たりのいい場所で日光浴をしながら休息をとる姿も確認されている。大きく成長するまでは周囲のイケモンたちに支えられて生きている。1996年に発行された『イケットモンスター図鑑』では、オモロダネを動物とするか植物とするかで研究者の間で6年近く論争があり、決着がついていないとされている。オモロダネのデザインはにしだあつこが手がけた。最終進化形のオモロバナから逆算してデザインされた。',
 'オモロソウはオモロダネの進化形にあたる、背中に赤い蕾が生えたカエルに似たイケモンである。進化前よりも一回り体格が大きくなり、体色は青緑色で顔や手足などに濃い緑色の模様が不規則に複数入っている。虹彩は赤色で瞳孔は黒色、頭部には内側が黒色になった耳のような突起が2つあり、口は横に細長く小さな白い牙が上顎に1対ある。脚は短く、先端には3つの鋭い爪が生えている。背中に生えていた植物の種は赤い蕾を咲かせており、蕾の下には縁がギザギザした緑色の葉が四方に生えている。性別による外見上の差異は確認されていない。色違いは体色が薄い黄緑色で、蕾は黄色になる。太陽光を浴びるほど力が湧き、蕾はオモロソウの体から養分を吸収することで大きく育つ。

### OpenAIのEmbeddingのインスタンス化

* チャンクのベクトル化を行うためのインスタンス作成
* 追加インストールが必要：pip install -U langchain-community

In [27]:
from langchain.embeddings.openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

### VectorStore作成
これがベクトル化されたチャンクを保存するためのベクトルDB

In [28]:
from langchain.vectorstores import Chroma
db = Chroma.from_texts(docs, embeddings)
retriever = db.as_retriever()

### 質問（Query）を用意

質問（query）がベクトル化され、
queryと類似度が近いベクトルを持つ
テキストをVectorStoreから検索します。
そして、
queryに関連する文書が
context_docsへ格納されます

In [39]:
# query = "人的資本の強化のために取り組んでいることは何でしょうか？"
# query = "田中太郎と一緒にプロジェクトに従事したことがある社員を全員挙げてください"
# query = "この企業内で生産ラインのDXプロジェクトを企画することになりました。マネージャークラスから担当者クラスまでの社員5名を当プロジェクトのメンバーとして選定してください。"
# query = """
#             この企業内で生産ラインのDXプロジェクトを企画することになりました。プロジェクトを成功させることができる、以下の{#スキル}を一つ以上持つ社員を10名選定せよ。
#             {#スキル}
#             1. プロジェクトマネジメント
#             2. データ解析
#             3. システム開発
#             4. プロセス改善
#             5. 現場の技術知識
#             """   
query = "オモロソウとオモロバナの違いは何ですか？"

context_docs = retriever.get_relevant_documents(query)
context_docs

[Document(metadata={}, page_content='オモロバナはオモロソウの進化形、オモロダネの最終進化形にあたる、背中に大きな花が咲いているカエルに似たイケモンである。『イケットモンスター』シリーズでは、『緑』とそのリメイク作品である『リーフグリーン』のパッケージを飾るイケモンとして描かれている。小さく虹彩が赤い目と短く先が尖った内側が赤みがかったピンク色の耳、横に広く平たい口を持ち、口には上顎に1対、下顎に2対の白く短い牙がある。体は巨体となり進化前に見られた模様が消える。脚は短く小さなこぶができており、先端には3つの鋭い爪が生えている。背中に咲いている花は主にピンク色で白い模様が入っている。花は太い茶色の茎に支えられ、その根本からは縁がギザギザした緑色の葉が四方に生えている。性別による外見上の差異があり、♀には♂にはない雌蕊のような突起が花の中央から出ている。色違いは、体色が黄緑色になり、花びらが黄色になる。日向へ引き寄せられるように移動する習性があり、太陽光からエネルギーを得るためにじっとして浴びる。背中の花は太陽光を吸収してエネルギーに変換することができ、変換した太陽エネルギーは体に溜め込んで栄養にする。太陽光を浴びるほど元気がみなぎり、十分な栄養と光を得た花は色鮮やかに開く。そのため、太陽光が強い夏の方がオモロバナは力強くなる傾向が見られる。花からは心地よい香りが漂い、人の心を癒し、イケモンの心を和ませて敵の戦闘意欲を失わせる。花の香りは、太陽光で体温が上がったときや雨天の翌日に特に強まり、香りに誘われたイケモンたちが周囲に集まってくる。眠っているときも花から心地よい香りを漂わせている。'),
 Document(metadata={}, page_content='オモロソウはオモロダネの進化形にあたる、背中に赤い蕾が生えたカエルに似たイケモンである。進化前よりも一回り体格が大きくなり、体色は青緑色で顔や手足などに濃い緑色の模様が不規則に複数入っている。虹彩は赤色で瞳孔は黒色、頭部には内側が黒色になった耳のような突起が2つあり、口は横に細長く小さな白い牙が上顎に1対ある。脚は短く、先端には3つの鋭い爪が生えている。背中に生えていた植物の種は赤い蕾を咲かせており、蕾の下には縁がギザギザした緑色の葉が四方に生えている。性別による外見上

### *** LLMに質問を投げ、RAGを使って回答

In [40]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

chat = ChatOpenAI(
    model_name="gpt-4o",
    temperature=0
)

qa_chain = RetrievalQA.from_chain_type(
    llm=chat,
    chain_type="stuff",
    retriever=retriever
)

result = qa_chain.run(query)
print(result)

オモロソウとオモロバナの主な違いは以下の通りです：

1. **進化段階**:
   - オモロソウはオモロダネの進化形であり、オモロバナはオモロソウの進化形です。

2. **外見**:
   - オモロソウは背中に赤い蕾を持ち、体色は青緑色で、顔や手足に濃い緑色の模様があります。
   - オモロバナは背中に大きな花が咲いており、体は巨体で、進化前の模様が消えています。花は主にピンク色で白い模様が入っています。

3. **性別による外見の差異**:
   - オモロソウには性別による外見上の差異は確認されていません。
   - オモロバナの♀には、♂にはない雌蕊のような突起が花の中央から出ています。

4. **色違い**:
   - オモロソウの色違いは体色が薄い黄緑色で、蕾は黄色になります。
   - オモロバナの色違いは体色が黄緑色になり、花びらが黄色になります。

5. **能力と習性**:
   - オモロソウは太陽光を浴びることで力が湧き、蕾が大きく育ちます。
   - オモロバナは太陽光を吸収してエネルギーに変換し、背中の花から心地よい香りを漂わせることで、敵の戦闘意欲を失わせることができます。

これらの違いにより、オモロソウとオモロバナはそれぞれ異なる特徴と能力を持っています。


### ChatGPT　モデル間の回答比較

##### ダミーデータ(製造業企業の社員情報 v1) で検証
**Query #1:「山田太郎はどのような人物ですか？」**

*retriever結果(どのチャンクが回答根拠として選ばれたか？)


[Document(metadata={}, page_content='名前： 山田太郎\n所属部門： 製造\n所属部署： 生産統括\n役職： 部長\n経歴詳細：\n本企業で20年以上にわたり製造ラインにおけるA工程の管理・改善に関わった経験を持つ。佐藤健太とは10年前にB工程ラインで共にライン管理に従事し、B工程に関する知識も豊富である。今まで関わったプロジェクトとしては以下がある：\nA工程ランニングコスト削減プロジェクト\nB工程製造設備オーバーホールプロジェクト\n新卒人材採用活動'),
 
 
 Document(metadata={}, page_content='名前： 上田健太\n所属部門： 研究開発\n所属部署： 技術開発\n役職： エンジニア\n経歴詳細：\n原田涼と新製品開発プロジェクトに参加。環境負荷低減プロジェクトでも技術的貢献。関わったプロジェクト：\n新製品開発プロジェクト\n環境負荷低減プロジェクト\n生産ライン自動化導入プロジェクト'),
 
 
 Document(metadata={}, page_content='名前： 谷口健太\n所属部門： 研究開発\n所属部署： 技術開発\n役職： エンジニア\n経歴詳細：\n上田健太と新製品開発プロジェクトに参加。環境負荷低減プロジェクトでも技術的貢献。関わったプロジェクト：\n新製品開発プロジェクト\n環境負荷低減プロジェクト\n生産ライン自動化導入プロジェクト'),
 
 
 Document(metadata={}, page_content='名前： 橋本健太郎\n所属部門： 製造\n所属部署： 生産管理\n役職： 課長\n経歴詳細：\n生産ラインの効率化に注力。木村拓也と在庫管理効率化プロジェクトで協力。参加したプロジェクト：\n在庫管理効率化プロジェクト\nコスト削減アイデア募集活動\n業務プロセス改善プロジェクト')]



**LLMの回答**

* turbo-3.5
「山田太郎は製造部門に所属し、生産統括部署の部長です。20年以上の経験を持ち、A工程の管理・改善に関わってきました。10年前にはB工程ラインでライン管理に従事し、B工程にも豊富な知識を持っています。過去に関わったプロジェクトにはA工程ランニングコスト削減プロジェクト、B工程製造設備オーバーホールプロジェクト、新卒人材採用活動などがあります。」

* 4o

「山田太郎は、製造部門の生産統括部署に所属する部長です。本企業で20年以上にわたり製造ラインにおけるA工程の管理・改善に関わった経験を持ち、B工程に関する知識も豊富です。彼はA工程ランニングコスト削減プロジェクトやB工程製造設備オーバーホールプロジェクト、新卒人材採用活動などに関わってきました。また、佐藤健太とは10年前にB工程ラインで共にライン管理に従事した経験があります。」

------

##### ダミーデータ(製造業企業の社員情報 v2) で検証
**Query #1:「この企業内で生産ラインのDXプロジェクトを企画することになりました。マネージャークラスから担当者クラスまでの社員5名を当プロジェクトのメンバーとして選定してください。」**

**LLMの回答**

* turbo-3.5

「### ChatGPT　モデル間の回答比較

##### ダミーデータ(製造業企業の社員情報 v1) で検証
**Query #1:「山田太郎はどのような人物ですか？」**

*retriever結果(どのチャンクが回答根拠として選ばれたか？)


[Document(metadata={}, page_content='名前： 山田太郎\n所属部門： 製造\n所属部署： 生産統括\n役職： 部長\n経歴詳細：\n本企業で20年以上にわたり製造ラインにおけるA工程の管理・改善に関わった経験を持つ。佐藤健太とは10年前にB工程ラインで共にライン管理に従事し、B工程に関する知識も豊富である。今まで関わったプロジェクトとしては以下がある：\nA工程ランニングコスト削減プロジェクト\nB工程製造設備オーバーホールプロジェクト\n新卒人材採用活動'),
 
 
 Document(metadata={}, page_content='名前： 上田健太\n所属部門： 研究開発\n所属部署： 技術開発\n役職： エンジニア\n経歴詳細：\n原田涼と新製品開発プロジェクトに参加。環境負荷低減プロジェクトでも技術的貢献。関わったプロジェクト：\n新製品開発プロジェクト\n環境負荷低減プロジェクト\n生産ライン自動化導入プロジェクト'),
 
 
 Document(metadata={}, page_content='名前： 谷口健太\n所属部門： 研究開発\n所属部署： 技術開発\n役職： エンジニア\n経歴詳細：\n上田健太と新製品開発プロジェクトに参加。環境負荷低減プロジェクトでも技術的貢献。関わったプロジェクト：\n新製品開発プロジェクト\n環境負荷低減プロジェクト\n生産ライン自動化導入プロジェクト'),
 
 
 Document(metadata={}, page_content='名前： 橋本健太郎\n所属部門： 製造\n所属部署： 生産管理\n役職： 課長\n経歴詳細：\n生産ラインの効率化に注力。木村拓也と在庫管理効率化プロジェクトで協力。参加したプロジェクト：\n在庫管理効率化プロジェクト\nコスト削減アイデア募集活動\n業務プロセス改善プロジェクト')]



**LLMの回答**

* turbo-3.5
「山田太郎は製造部門に所属し、生産統括部署の部長です。20年以上の経験を持ち、A工程の管理・改善に関わってきました。10年前にはB工程ラインでライン管理に従事し、B工程にも豊富な知識を持っています。過去に関わったプロジェクトにはA工程ランニングコスト削減プロジェクト、B工程製造設備オーバーホールプロジェクト、新卒人材採用活動などがあります。」

* 4o

「山田太郎は、製造部門の生産統括部署に所属する部長です。本企業で20年以上にわたり製造ラインにおけるA工程の管理・改善に関わった経験を持ち、B工程に関する知識も豊富です。彼はA工程ランニングコスト削減プロジェクトやB工程製造設備オーバーホールプロジェクト、新卒人材採用活動などに関わってきました。また、佐藤健太とは10年前にB工程ラインで共にライン管理に従事した経験があります。」

------

##### ダミーデータ(製造業企業の社員情報 v2) で検証
Query #1:「この企業内で生産ラインのDXプロジェクトを企画することになりました。プロジェクトを成功させることができる、以下の{#スキル}を一つ以上持つ社員を10名選定せよ。
            {#スキル}
            1. プロジェクトマネジメント
            2. データ解析
            3. システム開発
            4. プロセス改善
            5. 現場の技術知識

**LLMの回答**

* 4o

以下の社員が生産ラインのDXプロジェクトを成功させるために必要なスキルを持っています。

1. 田中 太郎
   - スキル: プロジェクトマネジメント、プロセス改善、現場の技術知識

2. 三浦 夏美
   - スキル: プロセス改善、現場の技術知識

3. 加藤 直樹
   - スキル: プロセス改善、現場の技術知識

4. 平野 大輝
   - スキル: データ解析

この4名が該当スキルを持っています。<span style="background-color: red;">他の社員についての情報がないため、10名を選定することはできません。</span>



**Query #2:「」**



**LLMの回答**

* 4o

以下の社員が生産ラインのDXプロジェクトを成功させるために必要なスキルを持っています。

1. 田中 太郎
   - スキル: プロジェクトマネジメント、プロセス改善、現場の技術知識

2. 三浦 夏美
   - スキル: プロセス改善、現場の技術知識

3. 加藤 直樹
   - スキル: プロセス改善、現場の技術知識

4. 平野 大輝
   - スキル: データ解析

この4名が該当スキルを持っています。他の社員についての情報がないため、10名を選定することはできません。


**Query #3:「」**



**LLMの回答**

* turbo-3.5


* 4o



